
# 📊 Calculadora Muestral Interactiva (Colab)

**Autor:** Nilton Yanac + ChatGPT — *Data Scientist & Muestreo*  
**Fecha:** 2025-09-15

Esta notebook te permite **iterar** sobre los parámetros clave de una fórmula de tamaño muestral y **visualizar su impacto** de forma inmediata:

- Tamaño de población \(N\) (con corrección por población finita, FPC)
- Margen de error \(e\)
- Nivel de confianza \(\Rightarrow Z\)
- Proporción esperada \(p\) (o varianza poblacional para peor caso \(p=0.5\))
- **Design Effect (deff)** para diseños complejos
- **Tasa de respuesta** (o factor de no respuesta)

Incluye:
- UI con sliders (ipywidgets)
- Cálculo paso a paso (n∞, FPC, deff, ajuste por no respuesta)
- **Gráficos de sensibilidad** por variable
- **Cuadrícula de escenarios** y exportación a CSV

> Fórmula base (población infinita):  
> \[ n_0 = \frac{Z^2 \; p (1-p)}{e^2} \]

> Corrección por población finita (si \(N\) es finito):  
> \[ n_{FPC} = \frac{n_0}{1 + \frac{n_0 - 1}{N}} \]

> Ajustes:  
> \[ n_{deff} = n_{FPC} \cdot \text{deff} \quad ; \quad n_{final} = \frac{n_{deff}}{\text{tasa\_respuesta}} \]

---

### Cómo usar
1. Ejecuta las celdas en orden (o desde `Entorno de ejecución > Ejecutar todo`).
2. En la sección **UI interactiva**, mueve los *sliders* para ver el impacto.
3. Revisa los **gráficos de sensibilidad**.
4. Genera tu **cuadrícula de escenarios** y descárgala en CSV.

> **Sugerencia:** si no conoces \(p\), usa \(p=0.5\) (peor caso).



## 🔧 Instalación rápida (si es necesario en Colab)

> Normalmente **Colab ya trae ipywidgets**. Si notas que los controles no aparecen,
> descomenta y ejecuta estas líneas:

```python
# !pip install ipywidgets
# from google.colab import output
# output.enable_custom_widget_manager()
```


In [ ]:

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    import ipywidgets as widgets
    from IPython.display import display, Markdown, clear_output
except Exception as e:
    print("Advertencia: ipywidgets no disponible. En Colab, ejecuta la celda de instalación si no ves los sliders.")
    from IPython.display import display, Markdown, clear_output


In [ ]:

# =============================
#   Funciones de utilidad
# =============================

# Tabla de niveles de confianza -> Z (aprox.)
Z_TABLE = {
    0.80: 1.2816,
    0.85: 1.4395,
    0.90: 1.6449,
    0.95: 1.96,
    0.975: 2.2414,  # 97.5%
    0.98: 2.3263,
    0.99: 2.5758,
    0.995: 2.8070,
    0.999: 3.2905
}

def z_from_conf(conf: float) -> float:
    # Si coincide con tabla:
    if conf in Z_TABLE:
        return Z_TABLE[conf]
    # Interpolación lineal entre puntos de la tabla
    xs = sorted(Z_TABLE.keys())
    if conf <= xs[0]:
        return Z_TABLE[xs[0]]
    if conf >= xs[-1]:
        return Z_TABLE[xs[-1]]
    for i in range(len(xs)-1):
        if xs[i] <= conf <= xs[i+1]:
            x0, x1 = xs[i], xs[i+1]
            y0, y1 = Z_TABLE[x0], Z_TABLE[x1]
            t = (conf - x0) / (x1 - x0)
            return y0 + t * (y1 - y0)

def n0_infinite(p: float, e: float, conf: float) -> float:
    Z = z_from_conf(conf)
    return (Z**2 * p * (1 - p)) / (e**2)

def apply_fpc(n0: float, N: float) -> float:
    if np.isinf(N):
        return n0
    if N <= 0:
        return n0
    return n0 / (1 + (n0 - 1) / N)

def apply_deff(n: float, deff: float) -> float:
    return n * deff

def apply_response_rate(n: float, rr: float) -> float:
    rr = max(min(rr, 1.0), 1e-6)  # evita división por cero
    return n / rr

def sample_size(N: float, e: float, conf: float, p: float, deff: float = 1.0, rr: float = 1.0):
    # Retorna un dict con n0, n_fpc, n_deff, n_final, Z
    Z = z_from_conf(conf)
    n0 = n0_infinite(p, e, conf)
    n_fpc = apply_fpc(n0, N)
    n_deff = apply_deff(n_fpc, deff)
    n_final = apply_response_rate(n_deff, rr)
    return {
        "Z": Z,
        "n0_infinita": n0,
        "n_fpc": n_fpc,
        "n_ajustada_deff": n_deff,
        "n_final_ajustada_no_respuesta": n_final
    }


In [ ]:

# =============================
#   UI Interactiva (ipywidgets)
# =============================

style = {"description_width": "180px"}
layout = widgets.Layout(width="450px")

N_slider = widgets.FloatLogSlider(
    value=1e5, base=10, min=3, max=8, step=0.01,
    description="Población N (log):", style=style, layout=layout,
    readout=True, readout_format='.0f'
)

infinite_toggle = widgets.Checkbox(
    value=False, description="Usar N infinita (sin FPC)"
)

e_slider = widgets.FloatSlider(
    value=0.05, min=0.002, max=0.2, step=0.001,
    description="Margen de error e:", style=style, layout=layout,
    readout_format=".3f"
)

conf_slider = widgets.FloatSlider(
    value=0.95, min=0.80, max=0.999, step=0.001,
    description="Nivel de confianza:", style=style, layout=layout,
    readout_format=".3f"
)

p_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.001,
    description="Proporción esperada p:", style=style, layout=layout,
    readout_format=".3f"
)

deff_slider = widgets.FloatSlider(
    value=1.0, min=1.0, max=4.0, step=0.01,
    description="Design effect (deff):", style=style, layout=layout,
    readout_format=".2f"
)

rr_slider = widgets.FloatSlider(
    value=0.8, min=0.1, max=1.0, step=0.01,
    description="Tasa de respuesta (rr):", style=style, layout=layout,
    readout_format=".2f"
)

out = widgets.Output()

def fmt(n):
    if n < 1000:
        return f"{n:,.1f}"
    return f"{n:,.0f}"

def on_change(_=None):
    with out:
        clear_output()
        N_val = np.inf if infinite_toggle.value else N_slider.value
        res = sample_size(
            N=N_val,
            e=e_slider.value,
            conf=conf_slider.value,
            p=p_slider.value,
            deff=deff_slider.value,
            rr=rr_slider.value
        )
        display(Markdown("### ✅ Resultado"))
        print(f"Z (según confianza): {res['Z']:.4f}")
        print(f"n0 (población infinita): {fmt(res['n0_infinita'])}")
        if np.isfinite(N_val):
            print(f"n con FPC (N finita): {fmt(res['n_fpc'])}")
        else:
            print(f"n con FPC (N infinita): {fmt(res['n_fpc'])}")
        print(f"n ajustada por deff: {fmt(res['n_ajustada_deff'])}")
        print(f"n final ajustada por no respuesta: {fmt(res['n_final_ajustada_no_respuesta'])}")

        # Gráficos de sensibilidad (uno por figura)
        # 1) Sensibilidad vs margen de error
        es = np.linspace(0.005, 0.15, 80)
        ns_e = [sample_size(N_val, e, conf_slider.value, p_slider.value, deff_slider.value, rr_slider.value)["n_final_ajustada_no_respuesta"] for e in es]
        plt.figure()
        plt.plot(es, ns_e)
        plt.xlabel("Margen de error e")
        plt.ylabel("n final")
        plt.title("Sensibilidad: n vs e")
        plt.grid(True)
        plt.show()

        # 2) Sensibilidad vs p
        ps = np.linspace(0.01, 0.99, 80)
        ns_p = [sample_size(N_val, e_slider.value, conf_slider.value, p, deff_slider.value, rr_slider.value)["n_final_ajustada_no_respuesta"] for p in ps]
        plt.figure()
        plt.plot(ps, ns_p)
        plt.xlabel("Proporción p")
        plt.ylabel("n final")
        plt.title("Sensibilidad: n vs p")
        plt.grid(True)
        plt.show()

        # 3) Sensibilidad vs confianza
        cs = np.linspace(0.80, 0.999, 80)
        ns_c = [sample_size(N_val, e_slider.value, c, p_slider.value, deff_slider.value, rr_slider.value)["n_final_ajustada_no_respuesta"] for c in cs]
        plt.figure()
        plt.plot(cs, ns_c)
        plt.xlabel("Nivel de confianza")
        plt.ylabel("n final")
        plt.title("Sensibilidad: n vs confianza")
        plt.grid(True)
        plt.show()

        # 4) Sensibilidad vs deff
        ds = np.linspace(1.0, 4.0, 80)
        ns_d = [sample_size(N_val, e_slider.value, conf_slider.value, p_slider.value, d, rr_slider.value)["n_final_ajustada_no_respuesta"] for d in ds]
        plt.figure()
        plt.plot(ds, ns_d)
        plt.xlabel("Design effect (deff)")
        plt.ylabel("n final")
        plt.title("Sensibilidad: n vs deff")
        plt.grid(True)
        plt.show()

        # 5) Sensibilidad vs tasa de respuesta
        rrs = np.linspace(0.1, 1.0, 80)
        ns_rr = [sample_size(N_val, e_slider.value, conf_slider.value, p_slider.value, deff_slider.value, r)["n_final_ajustada_no_respuesta"] for r in rrs]
        plt.figure()
        plt.plot(rrs, ns_rr)
        plt.xlabel("Tasa de respuesta (rr)")
        plt.ylabel("n final")
        plt.title("Sensibilidad: n vs tasa de respuesta")
        plt.grid(True)
        plt.show()

        # 6) Sensibilidad vs N
        Ns = np.logspace(3, 8, 80)
        ns_N = [sample_size(N, e_slider.value, conf_slider.value, p_slider.value, deff_slider.value, rr_slider.value)["n_final_ajustada_no_respuesta"] for N in Ns]
        plt.figure()
        plt.semilogx(Ns, ns_N)
        plt.xlabel("Población N (escala log)")
        plt.ylabel("n final")
        plt.title("Sensibilidad: n vs N")
        plt.grid(True, which="both")
        plt.show()

controls = widgets.VBox([
    widgets.HBox([N_slider, widgets.VBox([infinite_toggle])]),
    e_slider, conf_slider, p_slider, deff_slider, rr_slider
])

for w in [N_slider, infinite_toggle, e_slider, conf_slider, p_slider, deff_slider, rr_slider]:
    w.observe(on_change, names='value')

display(Markdown("## 🎛️ Controles"))
display(controls)
display(out)

on_change()


## 🎛️ Controles

Output()


## 🧪 Cuadrícula de escenarios y exportación

La celda siguiente genera una **tabla de escenarios** variando márgenes de error, niveles de confianza, proporciones, tamaños de población, design effect y tasas de respuesta. Úsala para comparar configuraciones.

> Edita las listas `ES`, `CONFS`, `PS`, `NS`, `DEFFS`, `RRS` para personalizar tu grilla.


In [ ]:

from itertools import product

# Define aquí tus mallas
ES = [0.03, 0.05, 0.07]
CONFS = [0.90, 0.95, 0.99]
PS = [0.5, 0.3, 0.1]
NS = [np.inf, 5_000, 50_000, 500_000]
DEFFS = [1.0, 1.5]
RRS = [0.7, 0.85, 1.0]

rows = []
for e, conf, p, N, deff, rr in product(ES, CONFS, PS, NS, DEFFS, RRS):
    res = sample_size(N, e, conf, p, deff, rr)
    rows.append({
        "e": e,
        "conf": conf,
        "p": p,
        "N": N if np.isfinite(N) else "inf",
        "deff": deff,
        "rr": rr,
        "Z": round(res["Z"], 4),
        "n0": res["n0_infinita"],
        "n_fpc": res["n_fpc"],
        "n_deff": res["n_ajustada_deff"],
        "n_final": res["n_final_ajustada_no_respuesta"]
    })

df_scenarios = pd.DataFrame(rows)
df_scenarios["n0"] = df_scenarios["n0"].round(2)
df_scenarios["n_fpc"] = df_scenarios["n_fpc"].round(2)
df_scenarios["n_deff"] = df_scenarios["n_deff"].round(2)
df_scenarios["n_final"] = df_scenarios["n_final"].apply(lambda x: int(np.ceil(x)))

df_scenarios.head(10)


,e,conf,p,N,deff,rr,Z,n0,n_fpc,n_deff,n_final
0,0.03,0.9,0.5,inf,1.0,0.70,1.6449,751.58,751.58,751.58,1074
1,0.03,0.9,0.5,inf,1.0,0.85,1.6449,751.58,751.58,751.58,885
2,0.03,0.9,0.5,inf,1.0,1.00,1.6449,751.58,751.58,751.58,752
3,0.03,0.9,0.5,inf,1.5,0.70,1.6449,751.58,751.58,1127.37,1611
4,0.03,0.9,0.5,inf,1.5,0.85,1.6449,751.58,751.58,1127.37,1327
5,0.03,0.9,0.5,inf,1.5,1.00,1.6449,751.58,751.58,1127.37,1128
6,0.03,0.9,0.5,5000,1.0,0.70,1.6449,751.58,653.48,653.48,934
7,0.03,0.9,0.5,5000,1.0,0.85,1.6449,751.58,653.48,653.48,769
8,0.03,0.9,0.5,5000,1.0,1.00,1.6449,751.58,653.48,653.48,654
9,0.03,0.9,0.5,5000,1.5,0.70,1.6449,751.58,653.48,980.23,1401


In [ ]:

# Exporta a CSV: en Colab será /content/, fuera de Colab guardará localmente
csv_path = "/content/sample_size_scenarios.csv"
try:
    df_scenarios.to_csv(csv_path, index=False)
    print(f"CSV exportado: {csv_path}")
except Exception as e:
    csv_path = "sample_size_scenarios.csv"
    df_scenarios.to_csv(csv_path, index=False)
    print(f"CSV exportado (local): {csv_path}")


CSV exportado: /content/sample_size_scenarios.csv
